In [8]:
pip install opencv-python

In [9]:
!git clone https://github.com/iml1111/opencv_pose_detection.git

Cloning into 'opencv_pose_detection'...
remote: Enumerating objects: 30, done.
remote: Total 30 (delta 0), reused 0 (delta 0), pack-reused 30
Unpacking objects: 100% (30/30), done.


In [10]:
cd /content/opencv_pose_detection/src/model

/content/opencv_pose_detection/src/model


In [11]:
ls coco.prototxt

coco.prototxt


In [17]:
# video
import cv2
import time
import numpy as np
import os

def proc(module, video):
    if module is "coco":
        protoFile = "/content/opencv_pose_detection/src/model/coco.prototxt"
        weightsFile = "/content/opencv_pose_detection/src/model/cooc.caffemodel"
        nPoints = 18
        POSE_PAIRS = [ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]

    elif module is "mpi":
        protoFile = "/content/opencv_pose_detection/src/model/mpi.prototxt"
        weightsFile = "/content/opencv_pose_detection/src/model/mpi.caffemodel"
        nPoints = 15
        POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]
    inWidth = 368
    inHeight = 368
    threshold = 0.1
    input_source = "/content/opencv_pose_detection/src/input/" + video
    print(input_source)
    cap = cv2.VideoCapture(input_source)

    hasFrame, frame = cap.read()
    vid_writer = cv2.VideoWriter('/content/opencv_pose_detection/src/output/' + video.split(".")[0] + "_"+ module + '.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))
    net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
    while cv2.waitKey(1) < 0:
        t = time.time()
        hasFrame, frame = cap.read()
        frameCopy = np.copy(frame)
        if not hasFrame:
            cv2.waitKey()
            break
        frameWidth = frame.shape[1]
        frameHeight = frame.shape[0]
        inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                                  (0, 0, 0), swapRB=False, crop=False)
        net.setInput(inpBlob)
        output = net.forward()
        H = output.shape[2]
        W = output.shape[3]
        points = []
        for i in range(nPoints):
            probMap = output[0, i, :, :]
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
            x = (frameWidth * point[0]) / W
            y = (frameHeight * point[1]) / H
            if prob > threshold : 
                cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
                points.append((int(x), int(y)))
            else :
                points.append(None)
        for pair in POSE_PAIRS:
            partA = pair[0]
            partB = pair[1]
            if points[partA] and points[partB]:
                cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3, lineType=cv2.LINE_AA)
                cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.circle(frame, points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.putText(frame, "Test Time = {:.2f} sec".format(time.time() - t), (50, 50), cv2.FONT_HERSHEY_COMPLEX, .8, (255, 50, 0), 2, lineType=cv2.LINE_AA)
        vid_writer.write(frame)
    vid_writer.release()

if __name__ == '__main__':
    module = ["mpi","coco"]
    for i in module:
        for j in os.listdir("/content/opencv_pose_detection/src/input"):
            if j[-3:] in ['mp4','avi']:
                proc(i, j)


/content/opencv_pose_detection/src/input/person01_handwaving.avi
/content/opencv_pose_detection/src/input/person01_handwaving.avi


error: ignored

In [14]:
# #image
# import cv2
# import time
# import numpy as np
# import sys
# import os

# def proc(module, img):
#     if module == "mpi":
#         protoFile = "/content/opencv_pose_detection/src/model/mpi.prototxt"
#         weightsFile = "/content/opencv_pose_detection/src/model/mpi.caffemodel"
#         nPoints = 15
#         POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13]]
#     else:
#         protoFile = "/content/opencv_pose_detection/src/model/coco.prototxt"
#         weightsFile = "/content/opencv_pose_detection/src/model/coco.caffemodel"
#         nPoints = 18
#         POSE_PAIRS = [[1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]
#     print("/content/opencv_pose_detection/src/input/" + img)
#     frame = cv2.imread("/content/opencv_pose_detection/src/input/" + img)
#     frameCopy = np.copy(frame)
#     frameWidth = frame.shape[1]
#     frameHeight = frame.shape[0]
#     threshold = 0.1
#     net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
#     t = time.time()
#     inWidth = 368
#     inHeight = 368
#     inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
#                               (0, 0, 0), swapRB=False, crop=False)
#     net.setInput(inpBlob)
#     output = net.forward()
#     print("완료 : {:.3f}".format(time.time() - t))
#     H = output.shape[2]
#     W = output.shape[3]
#     points = []
#     for i in range(nPoints):
#         probMap = output[0, i, :, :]
#         minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
#         x = (frameWidth * point[0]) / W
#         y = (frameHeight * point[1]) / H
#         if prob > threshold : 
#             cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
#             cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
#             points.append((int(x), int(y)))
#         else :
#             points.append(None)
#     for pair in POSE_PAIRS:
#         partA = pair[0]
#         partB = pair[1]
#         if points[partA] and points[partB]:
#             cv2.line(frame, points[partA], points[partB], (0, 255, 255), 2)
#             cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
#     cv2.imwrite('/content/opencv_pose_detection/src/output/' + module + "_" + img, frame)

# if __name__ == '__main__':
#     module = ["mpi","coco"]
#     for i in module:
#         for j in os.listdir("/content/opencv_pose_detection/src/input"):
#         	if j[-3:] in ['jpg','png','peg']:
#         		proc(i, j)


/content/opencv_pose_detection/src/input/000004812.jpg
완료 : 6.240
/content/opencv_pose_detection/src/input/000004812.jpg
완료 : 8.789
